### Export TSOSI data file


In [ ]:
from pathlib import Path
import sys
import os
import django

# Add the parent directory to the system path and setup django
BASE_DIR = str(Path(os.getcwd()).resolve().parent.parent)

if BASE_DIR not in sys.path:
    sys.path.append(BASE_DIR)

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "backend_site.settings")

django.setup()

from tsosi.data.preparation.doaj.publishers_202X import (
    get_config as doaj_publisher_config,
)
from tsosi.data.preparation.doaj.libraries_2024_now import (
    get_config as doaj_library_2024_config,
    get_fixture_config as doaj_library_fixture_2024_config,
)
from tsosi.data.preparation.doaj.libraries_2023 import (
    get_config as doaj_library_2023_config,
)
from tsosi.data.preparation.doaj.libraries_2021_2022 import (
    get_config as doaj_library_2021_2022_config,
)

year = 2024
file_path = (
    Path.home()
    / "Nextcloud/TSOSI_data/doaj/4_prepared/2025-01-07-DOAJ_Publisher_Report_2024_prepared.xlsx"
)
file_path = f"{BASE_DIR}/tsosi/data/fixtures/doaj_example.json"
sheet_name = "Sheet1"
config = doaj_library_fixture_2024_config(year, str(file_path))
config.get_data()
# config = doaj_library_2023_config(file_path, sheet_name)
config.generate_data_file()

### DEV


In [4]:
from pathlib import Path
import sys
import os
import django
import pandas as pd
from asgiref.sync import sync_to_async

# Add the parent directory to the system path and setup django
BASE_DIR = str(Path(os.getcwd()).resolve().parent.parent)

if BASE_DIR not in sys.path:
    sys.path.append(BASE_DIR)

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "backend_site.settings")

django.setup()

from tsosi.models import Entity


@sync_to_async
def run():

    data = Entity.objects.filter(
        country__isnull=False, is_recipient=False
    ).values("id", "country")
    df = pd.DataFrame.from_records(data)

    return df


res = await run()

In [ ]:
from pycountry import countries

d = (
    res.groupby("country")
    .count()
    .reset_index()
    .sort_values("id", ascending=False)
)
mapping = {c.alpha_2: c.name for c in countries}
d["country_name"] = d["country"].map(mapping)
d.rename(columns={"country": "country_code", "id": "number"}, inplace=True)


categories = [(50, "Upper"), (10, "Medium"), (0, "Lower")]

for c_threshold, c_name in sorted(categories, key=lambda x: x[0]):
    mask = d["number"] > c_threshold
    d.loc[mask, "category"] = c_name

d[["country_name", "country_code", "number", "category"]].to_excel(
    "supporters_extract.xlsx", index=False
)

In [14]:
sorted(categories, key=lambda x: x[0])

[(0, 'Lower'), (10, 'Medium'), (50, 'Upper')]

In [ ]:
from fontTools.ttLib.woff2 import compress
from pathlib import Path
import re

font_directory = Path.home() / "Documents/TSOSI/Graphisme/Mont/Mont"
otf_files = font_directory.glob("*.otf")
save_directory = Path.home() / "Documents/TSOSI/Fonts/Mont"

for file in otf_files:
    file_path_usefull = re.match(r".*(Mont[^\.]+)", file.name)
    save_path = (
        str(save_directory / file_path_usefull[1]).replace(" ", "_") + ".woff2"
    )
    compress(str(file), str(save_path))